In [36]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import requests
import re
from bs4 import BeautifulSoup

In [37]:
HOST = "http://localhost:1200"
# We need to add the stock ticker its specific news
FINVIZ = HOST + "/finviz/news/"
TEST_TICKER = FINVIZ + "AAPL"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36', 
    'Accept-Language': 'en-US,en;q=0.5', 
    'Accept-Encoding': 'gzip, deflate, br'
}

In [38]:
news_dict = []
r = requests.get(TEST_TICKER)
root = ET.fromstring(r.text)
for item in root.iter('item'):
    title = item.find('title').text
    description = item.find('description').text
    pub_date = None
    link = item.find('link').text
    author = item.find('author').text
    author = author.replace('(', '').replace(')', '')
    news = {
        'title': title, 
        'description': description,
        'pub_date':pub_date,
        'link':link,
        'author':author,
    }
    news_dict.append(news)

In [39]:
df = pd.DataFrame(data=news_dict)

In [40]:
# This method is a little slow for parsing the web
# but since there's embedded code in the paragraphs, svg, and other elements,
# it would be a nightmare to parse trough a custom method
links = df['link']
for index, row in df.iterrows():
    description = ''
    r = requests.get(row['link'], headers=headers)
    html = r.text
    soup = BeautifulSoup(html, 'lxml')
    content = soup.find('div', class_='caas-body')
    if content:
        for p in content.find_all('p'):
            p_text = p.string if p.string != None else ''
            description += p_text
        df.at[index, 'description'] = description
df

,title,description,pub_date,link,author
0,This week in tech: A banner week for Apple; wh...,By Louis JuricicInvesting.com -- Here is your ...,None,https://finance.yahoo.com/news/week-tech-banne...,Investing.com
1,Where Will Qualcomm Stock Be in 1 Year?,Qualcomm's (NASDAQ: QCOM) stock fell 6% on May...,None,https://finance.yahoo.com/m/9cc21796-c3af-3d96...,Motley Fool
2,Apple Threatens To Take a Bite Out of Regional...,"Online apps, led by Apple, provide high-intere...",None,https://finance.yahoo.com/m/5e26ac71-2c79-3da2...,Investopedia
3,"1 Semiconductor Stock That Could Join Apple, M...",United States Steel became the world's first $...,None,https://finance.yahoo.com/m/facbc8aa-f5d0-3898...,Motley Fool
4,Microsoft Stock: Bear vs. Bull,Investors just received a flood of fresh infor...,None,https://finance.yahoo.com/m/13a40a60-872f-3d1e...,Motley Fool
...,...,...,...,...,...
95,Apple Finds Strength in India as Overall Sales...,The company logged strong iPhone growth in pla...,None,https://finance.yahoo.com/m/3f0bf5b4-0363-3459...,The Wall Street Journal
96,Apple suffers back-to-back sales slump in cost...,Apple has reported its second straight drop in...,None,https://finance.yahoo.com/news/apple-suffers-b...,The Telegraph
97,"Apple Gains on Record Services Revenue, $90 Bi...",Apple shares gained after a strong second-quar...,None,https://finance.yahoo.com/m/7d137ca1-7c85-3029...,Investopedia
98,"Blocks Earnings Beat Expectations, Driven by C...",Shares have faced headwinds since March after ...,None,https://finance.yahoo.com/m/1c77e9f7-68a8-3558...,Barrons.com


In [41]:
df.to_excel(excel_writer='aapl.xlsx', index=False)